In [1]:
import pandas as pd
import glob
import os
import json
import sys
from collections import Counter
import numpy as np
from IPython.display import display_html 
pd.set_option('max_colwidth',200)
pd.set_option('max_rows',500)

# to show multiple tables side by side
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [3]:
imgs = pd.read_csv('../proc_data_phase0/spellchecking/all_responses_round0-3_cleaned.csv',sep="\t")

In [4]:
imgs.columns

Index(['Unnamed: 0', 'vg_img_id', 'cat', 'synset', 'vg_obj_name',
       'responses_r0', 'opt-outs', 'vg_domain', 'top_response_domain_r0',
       'responses_domains_r0', 'url', 'sample_type', 'responses_domains_r1',
       'responses_r1', 'top_response_domain_r1', 'responses_domains_r2',
       'responses_r2', 'top_response_domain_r2', 'responses_domains_r3',
       'responses_r3', 'top_response_domain_r3', 'vg_object_id',
       'all_responses', 'clean', 'canon', 'spellchecked'],
      dtype='object')

In [9]:
imgs.vg_obj_name.value_counts()

train           954
giraffe         915
shirt           904
bed             888
boy             853
horse           822
man             806
woman           766
cat             754
bench           714
table           687
desk            672
dog             654
girl            650
car             642
pizza           518
counter         516
plane           485
airplane        479
motorcycle      466
truck           465
zebra           461
jacket          451
boat            450
couch           366
chair           365
lady            342
house           340
guy             330
cow             324
carpet          307
bird            295
bridge          274
coat            267
cake            261
child           230
bowl            219
sheep           216
dress           190
bread           186
curtain         182
cabinet         154
sandwich        153
bun             143
sofa            141
rug             140
batter          110
cheese          110
jet             106
dugout           91


In [5]:
#pd.crosstab(df.top_response_domain_r1,df.top_response_domain_r2)

In [6]:
#pairs[pairs.totalfreq].sort_values(['relation','freq_both_orders'],ascending=[True, False])

In [7]:
df = pd.read_csv('domains_names_pairs_relations_v2.csv',index_col=0)
#b1=df.str.startswith('man')
df.columns=['domain', 'word1', 'word2', 'relation', 'totalfreq', 'freq_w1_w2',
       'freq_w2_w1', 'domain_w1', 'domain_w2', 'syn_w1', 'syn_w2']
df.head()
#df[df.word1=="man"].sort_values('totalfreq',ascending=False)
#df[(df.word1=="man") & (df.word2=="shirt")]
#df[df.freq_w2_w1 > df.freq_w1_w2]
#df[(df.relation=="not-covered")].sort_values('totalfreq',ascending=False).head(100)

,domain,word1,word2,relation,totalfreq,freq_w1_w2,freq_w2_w1,domain_w1,domain_w2,syn_w1,syn_w2
330,vehicles,airplane,plane,synonymy,15257,11326,3931,vehicles,vehicles,Synset('airplane.n.01'),Synset('airplane.n.01')
256,vehicles,motorcycle,bike,synonymy,2702,2673,29,vehicles,vehicles,Synset('motorcycle.n.01'),Synset('motorcycle.n.01')
331,vehicles,airplane,jet,hypernymy.1,1842,1670,172,vehicles,vehicles,Synset('airplane.n.01'),Synset('jet.n.01')
333,vehicles,plane,jet,hypernymy.1,1763,1488,275,vehicles,vehicles,Synset('airplane.n.01'),Synset('jet.n.01')
65,vehicles,truck,car,co-hyponymy,1564,1168,396,vehicles,vehicles,Synset('motor_vehicle.n.01'),Synset('motor_vehicle.n.01')


In [8]:
df[df.relation=="crossclassified"].domain.value_counts()

home              20286
people            12275
clothing           6833
buildings          6510
vehicles           5910
food               4231
animals_plants     3451
Name: domain, dtype: int64

In [9]:
df=pd.read_csv('pairs-annotation/annotated-crossclasssified-pairs-alldomains.csv',index_col=0)
df.head()
df.groupby(['domain','same_object'])['freq_w1_w2'].agg({'type': 'count', 'token': 'sum'}).unstack()
ft=pd.crosstab(df.domain,df.same_object,margins=True)#,normalize='index')
#f=df[df.same_object=='f']
display_side_by_side(ft)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


same_object,f,t,All
domain,,,
animals_plants,62,38,100
buildings,70,30,100
clothing,55,45,100
food,72,28,100
home,89,11,100
people,62,38,100
vehicles,75,25,100
All,485,215,700


In [10]:
f_all=df[df.same_object=='f'].subrelation.value_counts()
t_all=df[df.same_object=='t'].subrelation.value_counts()
print("false, subrelations:")
print("true, subrelations:")

print(f_all)
print(t_all)

false, subrelations:
true, subrelations:
adjacent      167
supports.1    122
part.2        102
supports.2     64
part.1         20
other          10
Name: subrelation, dtype: int64
co-hyponyms    148
overlapping     22
hypernym.1      19
hypernym.2      18
other            8
Name: subrelation, dtype: int64


In [11]:
#temp=df[df.subrelation!="other"]
temp=df
f=temp[temp.same_object=='f']
t=temp[temp.same_object=='t']
myt=pd.crosstab(f.domain,f.subrelation,margins=True)
myt.iloc[:, np.argsort(myt.loc['All'])]

subrelation,other,part.1,supports.2,part.2,supports.1,adjacent,All
domain,,,,,,,
animals_plants,3,1,3,6,2,47,62
buildings,2,3,2,19,1,43,70
clothing,0,0,24,0,3,28,55
food,0,1,14,25,11,21,72
home,0,7,2,8,63,9,89
people,0,0,16,0,42,4,62
vehicles,5,8,3,44,0,15,75
All,10,20,64,102,122,167,485


In [12]:
myt2=pd.crosstab(t.domain,t.subrelation,margins=True)
myt2.iloc[:, np.argsort(myt2.loc['All'])]

subrelation,other,hypernym.2,hypernym.1,overlapping,co-hyponyms,All
domain,,,,,,
animals_plants,0,4,2,11,21,38
buildings,1,0,4,5,20,30
clothing,0,3,8,0,34,45
food,2,5,1,6,14,28
home,2,0,0,0,9,11
people,1,3,0,0,34,38
vehicles,2,3,4,0,16,25
All,8,18,19,22,148,215


In [13]:
#for r in f.subrelation.unique():
#    print(r)
#    a=f[f.subrelation==r].sample(8)
#    print(a[['domain','word1','word2']])


In [14]:
pd.crosstab(t.domain_w1,t.domain_w2)

domain_w2,animals_plants,buildings,clothing,food,home,people,vehicles
domain_w1,,,,,,,
animals_plants,32,0,0,1,3,1,0
buildings,0,28,0,0,2,0,0
clothing,0,0,46,0,0,0,0
food,0,0,0,24,2,0,1
home,2,0,0,0,11,0,0
people,0,0,0,0,0,37,0
vehicles,0,0,0,0,0,0,25


In [15]:
pd.crosstab(f.domain_w1,f.domain_w2)

domain_w2,animals_plants,buildings,clothing,food,home,people,vehicles
domain_w1,,,,,,,
animals_plants,26,5,4,1,12,12,2
buildings,1,30,5,1,10,4,14
clothing,1,0,16,5,4,25,1
food,3,0,0,63,5,0,0
home,13,2,6,9,62,1,0
people,1,0,34,0,0,30,0
vehicles,4,7,3,0,5,0,58


In [17]:
f[f.subrelation=="adjacent"]

,domain,word1,word2,relation,same_object,subrelation,totalfreq,freq_w1_w2,freq_w2_w1,domain_w1,domain_w2,syn_w1,syn_w2
5459,animals_plants,horse,belt,crossclassified,f,adjacent,226,226,0,animals_plants,animals_plants,0,0
135,animals_plants,dog,cat,crossclassified,f,adjacent,219,111,108,animals_plants,animals_plants,0,0
823,animals_plants,giraffe,zebra,crossclassified,f,adjacent,197,120,77,animals_plants,animals_plants,0,0
4239,animals_plants,horse,shoe,crossclassified,f,adjacent,141,141,0,animals_plants,people,0,0
1113,animals_plants,cow,goat,crossclassified,f,adjacent,132,90,42,animals_plants,animals_plants,0,0
1103,animals_plants,sheep,cow,crossclassified,f,adjacent,114,66,48,animals_plants,animals_plants,0,0
5781,animals_plants,dog,belt,crossclassified,f,adjacent,106,106,0,animals_plants,animals_plants,0,0
1757,animals_plants,cow,horse,crossclassified,f,adjacent,99,75,24,animals_plants,animals_plants,0,0
1643,animals_plants,giraffe,camel,crossclassified,f,adjacent,94,94,0,animals_plants,animals_plants,0,0
3150,animals_plants,horse,t-shirt,crossclassified,f,adjacent,55,55,0,animals_plants,clothing,0,0
